In [5]:
import pandas as pd

data = pd.read_csv("/mnt/c/Users/POOYA/Desktop/data.csv")

data

,stationCode,Year_AB_1,Year_AB_2,Month_AB,Day_AB,BARAN_6,BARF_6,AB_BARF_6,BARAN_18,BARF_18,AB_BARF_18
0,110162090,1381,82,1,29,0.0,14,0.0,0.0,0.0,16.5
1,110162090,1381,82,2,2,0.0,0,0.0,5.0,0.0,0.0
2,110162090,1381,82,2,19,0.0,0,0.0,1.5,0.0,0.0
3,110162090,1381,82,2,28,0.0,0,0.0,2.0,0.0,0.0
4,110162090,1381,82,3,2,0.0,0,0.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3522,110164056,1397,98,3,9,3.0,0,0.0,0.0,0.0,0.0
3523,110164056,1397,98,3,10,0.0,0,0.0,3.0,0.0,0.0
3524,110164056,1397,98,3,11,3.0,0,0.0,0.0,0.0,0.0
3525,110164056,1397,98,3,12,1.0,0,0.0,0.0,0.0,0.0


In [6]:

def convertAB(Year_AB_1, Year_AB_2, Month_AB, Day_AB, Houre, Minute, Second):
    if Month_AB <=6 and Month_AB >=1:
        Year = Year_AB_2 + 1300
        Month = Month_AB
        Day = Day_AB
    elif Month_AB <=12 and Month_AB >=7:
        Year = Year_AB_1
        Month = Month_AB
        Day = Day_AB
    else:
        Year = None
        Month = None
        Day = None
    return [int(Year), int(Month), int(Day), int(Houre), int(Minute), int(Second)]



b6 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_6', 'BARF_6', 'AB_BARF_6']]
b6['Houre'] = 6
b6['Minute'] = 30
b6['Second'] = 0
b6['JAM_BARAN'] = b6['BARAN_6'] + b6['AB_BARF_6']

b6[['Date']] = b6.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b6 = b6[['stationCode', 'Date', 'BARAN_6', 'BARF_6', 'AB_BARF_6', 'JAM_BARAN']]
b6.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']



b18 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_18', 'BARF_18', 'AB_BARF_18']]
b18['Houre'] = 18
b18['Minute'] = 30
b18['Second'] = 0
b18['JAM_BARAN'] = b18['BARAN_18'] + b18['AB_BARF_18']

b18[['Date']] = b18.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b18 = b18[['stationCode', 'Date', 'BARAN_18', 'BARF_18', 'AB_BARF_18', 'JAM_BARAN']]
b18.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']




data = pd.concat([b6, b18])
data[['YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND']] = pd.DataFrame(data.DATE.tolist(), index= data.index)
data['DATE'] = data['YEAR'].astype(str) + "-" + data['MONTH'].astype(str).str.zfill(2) + "-" + data['DAY'].astype(str).str.zfill(2) + " " + data['HOURE'].astype(str).str.zfill(2) + ":" + data['MINUTE'].astype(str).str.zfill(2) + ":" + data['SECOND'].astype(str).str.zfill(2) 





data = data[['stationCode', 'DATE', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']]


data.sort_values(by=['stationCode', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND'], inplace=True)



<ipython-input-6-c0a81134442f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b6['Houre'] = 6
<ipython-input-6-c0a81134442f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b6['Minute'] = 30
<ipython-input-6-c0a81134442f>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [7]:
data

,stationCode,DATE,YEAR,MONTH,DAY,HOURE,MINUTE,SECOND,BARAN,BARF,AB_BARF,JAM_BARAN
1975,110147060,1360-09-10 06:30:00,1360,9,10,6,30,0,0.0,0.0,0.0,0.0
1975,110147060,1360-09-10 18:30:00,1360,9,10,18,30,0,99.9,0.0,0.0,99.9
1976,110147060,1360-10-10 06:30:00,1360,10,10,6,30,0,0.0,0.0,0.0,0.0
1976,110147060,1360-10-10 18:30:00,1360,10,10,18,30,0,0.9,0.0,0.0,0.9
1977,110147060,1360-11-17 06:30:00,1360,11,17,6,30,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3524,110164056,1398-03-11 18:30:00,1398,3,11,18,30,0,0.0,0.0,0.0,0.0
3525,110164056,1398-03-12 06:30:00,1398,3,12,6,30,0,1.0,0.0,0.0,1.0
3525,110164056,1398-03-12 18:30:00,1398,3,12,18,30,0,0.0,0.0,0.0,0.0
3526,110164056,1398-04-07 06:30:00,1398,4,7,6,30,0,0.0,0.0,0.0,0.0


In [8]:
data.to_csv("data.csv", index=False)